In [2]:
import os
import sys
import json
import numpy as np
import skimage.draw
import skimage.io
import shutil
import cv2
from google.colab.patches import cv2_imshow
import random
import json
import imutils

In [3]:
class LayoutDataset():

  def load_layout(self, dataset_dir, subset,passed_directory_name,page_name):
        """Load a subset of the Layout dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have 4 classes to add.
        #text','Equation','Table','Image'
        #self.add_class("layout", 1, "Text")
        #self.add_class("layout", 2, "Math")
        #self.add_class("layout", 3, "Table")
        #self.add_class("layout", 4, "Image")

        # Train or validation dataset?
        image_name_list = []
        #label_coord_dict = {}
        page_details_dict = {}
        table_coord_dict = {}
        image_coord_dict = {}
        math_coord_dict = {}
        text_coord_dict = {}
        label_annot_dict = {}
        #print(subset)
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        bbxs =[]
        list(map(bbxs.extend, [[bbx for  bbx in fls if  bbx.endswith(".bbx")  ]  for _,_,fls in os.walk(dataset_dir)  ]))
        #print(len(bbxs))
        #list(map(bounding_box_list.extend, [[bbx for  bbx in fls if  bbx.endswith(".bbx")  ]  for _,_,fls in os.walk(dataset_dir)  ]))
        for bb in bbxs:
            dir_name = bb.split('.')[0]
            #print(dir_name)
            if dir_name != passed_directory_name:
              continue
            annotations = json.load(open(os.path.join(dataset_dir, bb)))
            #val = annotations.values() # don't need the dict keys
            annotations_vals = list(annotations.values())
            imglst = list(annotations_vals[2].keys())
            #print(len(imglst))
            
            for i in range(len(imglst)):
            
                image_name = (list(imglst)[i])
                coords_yolo = []
                #print("************")
                #print(i)
                #print(image_name)
                
                annt_pp = annotations['images'][image_name]['annotations']#Annotation Per page
                #print(type(annt_pp))
                #print("annt_pp :",annt_pp)
                coords = [list(annt_pp[r]['bbox'].values() ) for r in range(len(annt_pp)) if 1 & 0 not in annt_pp[r]['bbox'].values() ]#co-ordinates for layout
                #print("coords :",coords)
                #print("coords = ",len(coords))
                #print(annt_pp)
                label = [(annt_pp[r]["label"]) for r in range(len(annt_pp)) if 1 & 0 not in annt_pp[r]['bbox'].values()]#label for layout
                #print("dir_name :",dir_name)
                #print(image_name,"=",label)
                #print("label :",label)

                #label_dict={'Text':0,'Math':1,'Table':2,'Image':3}
                temp_list = []
                for i in range(len(label)):
                  if label[i]=='Image':
                    image_name_list.append(image_name)
                    #print(dir_name+"/"+image_name+"#"+'Image',"=",coords[i])
                    image_coord_dict[dir_name+"/"+image_name+"_"+'Image'+"_"+str(i)] = coords[i]
                    label_annot_dict[dir_name+"/"+image_name+"_"+'Image'+"_"+str(i)] = annt_pp
                    temp_list.append(dir_name+"/"+image_name+"_"+'Image'+"_"+str(i))
                  elif label[i]=='Table':
                    table_coord_dict[dir_name+"/"+image_name+"_"+'Table'+"_"+str(i)] = coords[i]
                    label_annot_dict[dir_name+"/"+image_name+"_"+'Table'+"_"+str(i)] = annt_pp
                    temp_list.append(dir_name+"/"+image_name+"_"+'Table'+"_"+str(i))
                  elif label[i]=='Math':
                    math_coord_dict[dir_name+"/"+image_name+"_"+'Math'+"_"+str(i)] = coords[i]
                    label_annot_dict[dir_name+"/"+image_name+"_"+'Math'+"_"+str(i)] = annt_pp
                    temp_list.append(dir_name+"/"+image_name+"_"+'Math'+"_"+str(i))
                  elif label[i]=='Text':
                    text_coord_dict[dir_name+"/"+image_name+"_"+'Text'+"_"+str(i)] = coords[i]
                    label_annot_dict[dir_name+"/"+image_name+"_"+'Text'+"_"+str(i)] = annt_pp
                    temp_list.append(dir_name+"/"+image_name+"_"+'Text'+"_"+str(i))

                  if len(temp_list) > 0:
                       page_details_dict[image_name] = temp_list
                #label_ids = [label_dict[a] for a in label]
                #print(image_name,":",len(label))
                #filename = dir_name+"_"+image_name.split('.')[0]
                filename = image_name.split('.')[0]

                #image_list.append(filename)

                #for i in range(len(coords)):
                 # coords[i].append(label_ids[i])
                  
                #add_to_txt(filename+'.txt', coords,subset)
                #print(coords[0])
                #print(label_ids[0])
                
                #print(len(label)==len(coords))#cross check  no. of labels w.r.t coords
                
                image_path = os.path.join(dataset_dir,dir_name, image_name)
                #append_new_line('train.txt',image_path)
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]
                #print("height",height,"width",width)
                

                #add_to_txt(filename+'.txt', coords_yolo,subset)
                #print("coords_yolo:",coords_yolo)
                #print("coords_yolo = ",len(coords_yolo))
                #print("height:",height,"width:",width)
                #print(i)
                #print("############")
        return table_coord_dict,image_coord_dict,math_coord_dict,text_coord_dict,label_annot_dict,image_name_list,page_details_dict
                #break

In [4]:
dataset_path = 'Dataset'
dataset_train = LayoutDataset()

In [5]:
# LWK in the name of dictonaries does not signifies anything....   
table_coordinates_LWK,image_coordinates_LWK,math_coordinates_LWK,text_coordinates_LWK,label_annotation_LWK,image_list,page_details_LWK=dataset_train.load_layout(dataset_path, "train",'GON2','name')

In [6]:
import random

In [7]:
bb_coordinates = {}

In [8]:
def extract_roi(coord_list,page_image,img_name):
  
  img_name_modified = img_name.split('/')[0]+'_'+img_name.split('/')[1]
  print(img_name_modified)

  #page_name = random.choice(image_list)
  #start_cell = box_dict[num1]
  #end_cell = box_dict[num2]
  dh, dw, _ = page_image.shape
  x1 = int(coord_list[0]*dw)
  y1 = int(coord_list[2]*dh)
  x2 = int(coord_list[1]*dw)#end_cell[0]+end_cell[3]
  y2 = int(coord_list[3]*dh)#end_cell[1]+end_cell[2]

  # print(x," : ",l)
  print(x1,y1,x2,y2)
  bb_coordinates[img_name] = [x1,y1,x2,y2]
  #print(end_y)
  #table_image = imutils.resize(table_image,width = 2964) #previously it was 2564

  ROI = page_image[y1:y2, x1:x2]    
  # draw the bounding box on the image
  cv2.rectangle(page_image, (x1, y1), (x2, y2), (255,255,255), 2)
  cv2.imwrite(img_name_modified+'_extracted.png', ROI)

In [9]:
bb_coordinates

{}

In [10]:
import copy

In [11]:
bbx_dict ={"mask": '',
           "mask_name": "",
           "images":{
               
           } 
          }
page_dict = {}

In [12]:
# This function creates a bbx file for the newly generated page
def create_bbx(img_name,large_image,annot_list,small_image,img_for_bb):

      #print(annot_list)
      page_name = img_name
      dh,dw,_ = large_image.shape

      annot_list_copy = annot_list#copy.deepcopy(label_annotation_LWK[img_for_bb])
      #print(dh,dw)
      print("create_bbx = ",img_name," ","img_for_bb = ",img_for_bb)
      #print(bb_coordinates[small_image])
      page_dict[page_name] = {
            "attribution": "",
            "license": "",
            "license_url": "",
            }

      for i in range(len(annot_list_copy)):
        if annot_list_copy[i]['label']=='Image':
          if img_for_bb in image_coordinates_LWK:
            print("here")
            print(annot_list_copy[i]['bbox'])
            print('i = ',i)
            if annot_list_copy[i]['bbox']['xmin'] == image_coordinates_LWK[img_for_bb][0] and annot_list_copy[i]['bbox']['xmax'] == image_coordinates_LWK[img_for_bb][1] and annot_list_copy[i]['bbox']['ymin'] == image_coordinates_LWK[img_for_bb][2] and annot_list_copy[i]['bbox']['ymax'] == image_coordinates_LWK[img_for_bb][3]:
              annot_list_copy[i]['label'] = small_image.split('_')[2]
              annot_list_copy[i]['bbox']['xmin'] = bb_coordinates[img_for_bb][0]/dw
              annot_list_copy[i]['bbox']['ymin'] = bb_coordinates[img_for_bb][1]/dh
              annot_list_copy[i]['bbox']['xmax'] = bb_coordinates[img_for_bb][2]/dw
              annot_list_copy[i]['bbox']['ymax'] = bb_coordinates[img_for_bb][3]/dh
              print("**************")
              print(img_name)
              print("img_for_bb = ",img_for_bb," ","small_img = ",small_image)
              print("xmin : ",bb_coordinates[img_for_bb][0]/dw , "xmax : ",bb_coordinates[img_for_bb][2]/dw,"ymin : ",bb_coordinates[img_for_bb][1]/dh,"ymax : ",bb_coordinates[img_for_bb][3]/dh)
              print("updated label = ",small_image.split('_')[2])
              print("##############")
        elif annot_list_copy[i]['label']=='Table':
          if img_for_bb in table_coordinates_LWK:
            print(annot_list_copy[i]['bbox'])
            print('i = ',i)
            if annot_list_copy[i]['bbox']['xmin'] == table_coordinates_LWK[img_for_bb][0] and annot_list_copy[i]['bbox']['xmax'] == table_coordinates_LWK[img_for_bb][1] and annot_list_copy[i]['bbox']['ymin'] == table_coordinates_LWK[img_for_bb][2] and annot_list_copy[i]['bbox']['ymax'] == table_coordinates_LWK[img_for_bb][3]:
              annot_list_copy[i]['label'] = small_image.split('_')[2]
              print("%%%%%%%")
              print("hello")
              annot_list_copy[i]['bbox']['xmin'] = bb_coordinates[img_for_bb][0]/dw
              annot_list_copy[i]['bbox']['ymin'] = bb_coordinates[img_for_bb][1]/dh
              annot_list_copy[i]['bbox']['xmax'] = bb_coordinates[img_for_bb][2]/dw
              annot_list_copy[i]['bbox']['ymax'] = bb_coordinates[img_for_bb][3]/dh
              print("xmin : ",bb_coordinates[img_for_bb][0]/dw , "xmax : ",bb_coordinates[img_for_bb][2]/dw,"ymin : ",bb_coordinates[img_for_bb][1]/dh,"ymax : ",bb_coordinates[img_for_bb][3]/dh)
              print("updated label = ",small_image.split('_')[2])
              print("^^^^^^^^")
        elif annot_list_copy[i]['label']=='Math':
          if img_for_bb in math_coordinates_LWK:
            print(annot_list_copy[i]['bbox'])
            print('i = ',i)
            if annot_list_copy[i]['bbox']['xmin'] == math_coordinates_LWK[img_for_bb][0] and annot_list_copy[i]['bbox']['xmax'] == math_coordinates_LWK[img_for_bb][1] and annot_list_copy[i]['bbox']['ymin'] == math_coordinates_LWK[img_for_bb][2] and annot_list_copy[i]['bbox']['ymax'] == math_coordinates_LWK[img_for_bb][3]:
              annot_list_copy[i]['label'] = small_image.split('_')[2]
              print("%%%%%%%")
              print("Mathematics")
              annot_list_copy[i]['bbox']['xmin'] = bb_coordinates[img_for_bb][0]/dw
              annot_list_copy[i]['bbox']['ymin'] = bb_coordinates[img_for_bb][1]/dh
              annot_list_copy[i]['bbox']['xmax'] = bb_coordinates[img_for_bb][2]/dw
              annot_list_copy[i]['bbox']['ymax'] = bb_coordinates[img_for_bb][3]/dh
              print("xmin : ",bb_coordinates[img_for_bb][0]/dw , "xmax : ",bb_coordinates[img_for_bb][2]/dw,"ymin : ",bb_coordinates[img_for_bb][1]/dh,"ymax : ",bb_coordinates[img_for_bb][3]/dh)
              print("updated label = ",small_image.split('_')[2])
              print("^^^^^^^^")
        elif annot_list_copy[i]['label']=='Text':
          if img_for_bb in text_coordinates_LWK:
            print(annot_list_copy[i]['bbox'])
            print('i = ',i)
            if annot_list_copy[i]['bbox']['xmin'] == text_coordinates_LWK[img_for_bb][0] and annot_list_copy[i]['bbox']['xmax'] == text_coordinates_LWK[img_for_bb][1] and annot_list_copy[i]['bbox']['ymin'] == text_coordinates_LWK[img_for_bb][2] and annot_list_copy[i]['bbox']['ymax'] == text_coordinates_LWK[img_for_bb][3]:
              annot_list_copy[i]['label'] = small_image.split('_')[2]
              print("%%%%%%%")
              print("text")
              annot_list_copy[i]['bbox']['xmin'] = bb_coordinates[img_for_bb][0]/dw
              annot_list_copy[i]['bbox']['ymin'] = bb_coordinates[img_for_bb][1]/dh
              annot_list_copy[i]['bbox']['xmax'] = bb_coordinates[img_for_bb][2]/dw
              annot_list_copy[i]['bbox']['ymax'] = bb_coordinates[img_for_bb][3]/dh
              print("xmin : ",bb_coordinates[img_for_bb][0]/dw , "xmax : ",bb_coordinates[img_for_bb][2]/dw,"ymin : ",bb_coordinates[img_for_bb][1]/dh,"ymax : ",bb_coordinates[img_for_bb][3]/dh)
              print("updated label = ",small_image.split('_')[2])
              print("^^^^^^^^")
      page_dict[page_name]['annotations'] = annot_list_copy
      bbx_dict['images'] = page_dict

      annot_list_copy = {}

      #return bbx_dict

In [13]:
# THis function interchanges the size of the two images passed
def change_size_white(first_image,second_image):
  first_image_x = first_image.shape[0]
  first_image_y = first_image.shape[1]
  second_image_x = second_image.shape[0]
  second_image_y = second_image.shape[1]

  #reshape both images
  first_image = cv2.resize(first_image, (second_image_y, second_image_x))
  second_image = cv2.resize(second_image, (first_image_y, first_image_x)) 
  return first_image,second_image

In [14]:
def put_white_img(small_img_1,large_img_2,key_2,small_img_2):

  s_img_1 = small_img_1
  l_img_2 = large_img_2
  x_offset_1= bb_coordinates[key_2][0]
  y_offset_1= bb_coordinates[key_2][1]
  l_img_2[y_offset_1:y_offset_1+s_img_1.shape[0], x_offset_1:x_offset_1+s_img_1.shape[1]] = s_img_1

  return l_img_2

In [15]:
# This function handles Linear Tranformations like Rotation, Translation, Stretching, Scaling
def linear_transformation(large_img,key,small_img):

  lt_threshold = random.random()
  small_img_height,small_img_width,_ = small_img.shape
  transformed_small_img = small_img
  
  if lt_threshold > 0.3 and lt_threshold < 0.6:
     selection_threshold = random.random()

     if selection_threshold >= 0.3 and selection_threshold < 0.33:
       size_threshold = random.random()

       if size_threshold > 0.5:
         small_img_height = small_img_height + int(small_img_height*0.09)
         small_img_width = small_img_width + int(small_img_width*0.09)
         bb_coordinates[key][2] = small_img_width+bb_coordinates[key][0]
         bb_coordinates[key][3] = small_img_height+bb_coordinates[key][1]

       else:
         small_img_height = small_img_height - int(small_img_height*0.1)
         small_img_width = small_img_width - int(small_img_width*0.1)
         bb_coordinates[key][2] = small_img_width+bb_coordinates[key][0]
         bb_coordinates[key][3] = small_img_height+bb_coordinates[key][1]

       transformed_small_img = cv2.resize(small_img, (small_img_height, small_img_width))

     elif selection_threshold >= 0.33 and selection_threshold < 0.6:
       bb_coordinates[key][0] = bb_coordinates[key][0]+int(bb_coordinates[key][0]*0.02) # translation towards right
     else :
       bb_coordinates[key][0] = bb_coordinates[key][0]-int(bb_coordinates[key][0]*0.02) # translation towards left

  elif lt_threshold <= 0.3 :
    # invert (random 90 rotation)
    if abs(small_img_height-small_img_width) < 10:
      degree = random.randint(1,10)*90
      num_rows, num_cols = small_img.shape[:2]
      rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), degree, 1)
      transformed_small_img = cv2.warpAffine(small_img, rotation_matrix, (num_cols, num_rows),borderValue=(255,255,255))
     #= imutils.rotate(small_img, degree) 
    else:
      degree = random.randint(1,10)*180
      num_rows, num_cols = small_img.shape[:2]
      rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), degree, 1)
      transformed_small_img = cv2.warpAffine(small_img, rotation_matrix, (num_cols, num_rows),borderValue=(255,255,255))
    
  else :
    stretch_threshold = random.random()

    if stretch_threshold > 0.5:

      # width_wise stretch 
      small_img_width = small_img_width + int(small_img_width*0.07)
      dim = (small_img_width, small_img_height)
      bb_coordinates[key][2] = small_img_width+bb_coordinates[key][0] # icrementing x_max according to width
      transformed_small_img = cv2.resize(small_img, dim, interpolation = cv2.INTER_AREA)
    else:

      # height_wise stretch 
      small_img_height = small_img_height + int(small_img_height*0.07)
      dim = (small_img_width, small_img_height)
      bb_coordinates[key][3] = small_img_height+bb_coordinates[key][1] # icrementing y_max according to height
      transformed_small_img = cv2.resize(small_img, dim, interpolation = cv2.INTER_AREA)

  return transformed_small_img

In [16]:
# This function handles the replacement 
def replace(small_img_1,large_img_2,key_2,small_img_2):

  samll_img_1_aspect = float(small_img_1.shape[1])/small_img_1.shape[0]
  samll_img_2_aspect = float(small_img_2.shape[1])/small_img_2.shape[0]

  print("key_2 = ",key_2)

  min_size = min(small_img_1.size,small_img_2.size)
  max_size = max(small_img_1.size,small_img_2.size)
  perc = min_size*100/max_size

  s_img_1 = small_img_1
  l_img_2 = large_img_2
  x_offset_1= bb_coordinates[key_2][0]
  y_offset_1= bb_coordinates[key_2][1]
  l_img_2[y_offset_1:y_offset_1+s_img_1.shape[0], x_offset_1:x_offset_1+s_img_1.shape[1]] = s_img_1

  return l_img_2

In [17]:
new_dir = "generated_layout/dataset/train"
if not os.path.exists(new_dir):
    os.makedirs(new_dir)

In [18]:
len(image_list)

225

In [19]:
number = 0

In [35]:
# This function is used to update annotation after the splitting of text
def update_annotation(new_part_name,splitted_text_coordinates,part_of_page):

  text_number = int(part_of_page.split("_")[3])

  temp_dict = copy.deepcopy(label_annotation_LWK[part_of_page][text_number])

  temp_dict['bbox']['xmin'] = splitted_text_coordinates[0]#/743
  temp_dict['bbox']['ymin'] = splitted_text_coordinates[2]#/1247
  temp_dict['bbox']['xmax'] = splitted_text_coordinates[1]#/743
  temp_dict['bbox']['ymax'] = splitted_text_coordinates[3]#/1247

  #print(splitted_text_coordinates[2])
  #print(new_part_name)
  text_coordinates_LWK[new_part_name] = splitted_text_coordinates
  #text_coordinates_LWK[new_part_name][0] = splitted_text_coordinates[0]/743
  #text_coordinates_LWK[new_part_name][1] = splitted_text_coordinates[2]/743
  #text_coordinates_LWK[new_part_name][2] = splitted_text_coordinates[1]/1247
  #text_coordinates_LWK[new_part_name][3] = splitted_text_coordinates[3]/1247

  label_annotation_LWK[part_of_page].append(temp_dict)
  label_annotation_LWK[new_part_name] = label_annotation_LWK[part_of_page]

In [21]:
def add_padding(small_img_1,large_img_2,padding_coord,img_path):
  # for first small image
  s_img_1 = small_img_1
  #print("vnsdfiv",s_img_1.shape)
  l_img_2 = large_img_2
  x_offset_1= int(padding_coord[0]*743)
  y_offset_1= int(padding_coord[2]*1247)
  l_img_2[y_offset_1:y_offset_1+s_img_1.shape[0], x_offset_1:x_offset_1+s_img_1.shape[1]] = s_img_1
  cv2.imwrite(img_path,l_img_2)
  #cv2_imshow(l_img_2)

In [22]:
# This function divides the text into 3 parts of different sizes
def divide(page,part_of_page,img_path):
  
  splitted_text_coordinates = {}
  xmin = text_coordinates_LWK[part_of_page][0]
  xmax = text_coordinates_LWK[part_of_page][1]
  ymin = text_coordinates_LWK[part_of_page][2]
  ymax = text_coordinates_LWK[part_of_page][3]

  center_x = (xmin+xmax)/2
  second_center_x = (center_x+xmax)/2
  shifted_center_x = center_x-0.001
  shifted_second_center_x = second_center_x+0.001

  bbox_1 = [xmin,shifted_center_x-0.009,ymin,ymax]
  padding_1 = [shifted_center_x-0.001,center_x+0.001,ymin,ymax]
  bbox_2 = [center_x+0.009,second_center_x-0.009,ymin,ymax]
  padding_2 = [second_center_x-0.001,shifted_second_center_x+0.001,ymin,ymax]
  bbox_3 = [shifted_second_center_x+0.009,xmax,ymin,ymax] 

  padding_height = int((ymax-ymin)*1247)
  padding_1_width = int((center_x+0.001-shifted_center_x-0.001)*743)
  padding_2_width = int((shifted_second_center_x+0.001-second_center_x-0.001)*743)

  white_padding_1 = np.zeros((padding_height,25,3),dtype=np.uint8)
  white_padding_1.fill(255)

  white_padding_2 = np.zeros((padding_height,25,3),dtype=np.uint8)
  white_padding_2.fill(255)
  #print((padding_height,padding_2_width,3))
  #cv2_imshow(white_padding_2)

  add_padding(white_padding_1,page,padding_1,img_path)
  add_padding(white_padding_2,page,padding_2,img_path)

  splitted_text_coordinates[1] = bbox_1
  splitted_text_coordinates[2] = bbox_2
  splitted_text_coordinates[3] = bbox_3

  return splitted_text_coordinates 

In [36]:
def split_text(page_name,part_of_page):

  page = cv2.imread("Dataset/train/GON2/"+page_name)
  img_path = "Dataset/train/GON2/"+page_name
  #print(page.shape)
  
  splitted_text_coordinates = divide(page,part_of_page,img_path)
  #print(splitted_text_coordinates)

  for i in range(len(splitted_text_coordinates)):
    name = part_of_page+"_"+str(i+1)
    page_details_LWK[page_name].append(name)
    #print(name)
    update_annotation(name,splitted_text_coordinates[i+1],part_of_page)

  text_number = int(part_of_page.split("_")[3])
  label_annotation_LWK[part_of_page].pop(text_number)
  label_annotation_LWK.pop(part_of_page,None)
  page_details_LWK[page_name].remove(part_of_page)

In [ ]:
#page_name = random.choice(image_list)

In [24]:
#temp_page = page_details_LWK["GK2_page-0182.jpg"]
#for i in range(len(temp_page)):
#  if temp_page[i].split("_")[2] == 'Text':
      #print(temp_page[i],"=",text_coordinates_LWK[temp_page[i]])
#    width = int((text_coordinates_LWK[temp_page[i]][1]-text_coordinates_LWK[temp_page[i]][0])*743)
#    height = int((text_coordinates_LWK[temp_page[i]][1]-text_coordinates_LWK[temp_page[i]][0])*1247)
      #print(temp_page[i],"=",width,":",height)
#    if width >480:
#       split_threshold = random.random()
       # if split_threshold <= 0.15:
#       split_text("GK2_page-0182.jpg",temp_page[i])
#    else:
#       continue
#  break

In [37]:
for page_name in page_details_LWK.keys():
  temp_page = page_details_LWK[page_name]
  for i in range(len(temp_page)):
    if temp_page[i].split("_")[2] == 'Text':
      #print(temp_page[i],"=",text_coordinates_LWK[temp_page[i]])
      width = int((text_coordinates_LWK[temp_page[i]][1]-text_coordinates_LWK[temp_page[i]][0])*743)
      height = int((text_coordinates_LWK[temp_page[i]][1]-text_coordinates_LWK[temp_page[i]][0])*1247)
      #print(temp_page[i],"=",width,":",height)
      if width >500:
        split_threshold = random.random()
        if split_threshold <= 0.15:
          split_text(page_name,temp_page[i])
      else:
        continue
  #break

In [ ]:
text_coordinates_LWK['GON2/GK2_page-0184.jpg_Text_4']

[0.12570781426953567,
 0.4507361268403171,
 0.28951612903225804,
 0.31209677419354837]

In [ ]:
page_details_LWK['GK2_page-0184.jpg']

['GON2/GK2_page-0184.jpg_Text_0',
 'GON2/GK2_page-0184.jpg_Text_1',
 'GON2/GK2_page-0184.jpg_Text_2',
 'GON2/GK2_page-0184.jpg_Text_4',
 'GON2/GK2_page-0184.jpg_Image_5',
 'GON2/GK2_page-0184.jpg_Math_6',
 'GON2/GK2_page-0184.jpg_Math_7',
 'GON2/GK2_page-0184.jpg_Math_8',
 'GON2/GK2_page-0184.jpg_Math_9',
 'GON2/GK2_page-0184.jpg_Math_10',
 'GON2/GK2_page-0184.jpg_Math_11',
 'GON2/GK2_page-0184.jpg_Math_12',
 'GON2/GK2_page-0184.jpg_Math_13',
 'GON2/GK2_page-0184.jpg_Math_14',
 'GON2/GK2_page-0184.jpg_Text_3_1',
 'GON2/GK2_page-0184.jpg_Text_3_2',
 'GON2/GK2_page-0184.jpg_Text_3_3']

In [ ]:
from_random_page = page_details_LWK['GK2_page-0184.jpg'][0]

In [ ]:
text_count = 0
image_count = 0
table_count = 0
math_count = 0
total_count = len(label_annotation_LWK[from_random_page])
for i in range(total_count):
  j=label_annotation_LWK[from_random_page][i]['label']
  if j=='Text':
    text_count+=1
  elif j =='Image':
    image_count+=1
  elif j=='Table':
    table_count+=1
  else:
    math_count+=1

print("text_count = ",text_count)
print("image_count = ",image_count)
print("table_count = ",table_count)
print("math_count = ",math_count)
print("total_count = ",total_count)

text_count =  7
image_count =  1
table_count =  0
math_count =  9
total_count =  17


In [ ]:
to_replace_with,_ = random.choice(list(table_coordinates_LWK.items()))
print(to_replace_with)

GON2/GK2_page-0382.jpg_Table_15


In [ ]:
to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1]

'GON2/GK2_page-0382.jpg'

In [ ]:
to_replace_with.split("_")[2]

'Table'

In [26]:
def process_image(page_name,number):

  count = 0
  for part in page_details_LWK[page_name]:

    print(part+"&"+str(count))
    if count==0:
      main_img_page = cv2.imread(image_path+"/"+part.split("_")[0]+"_"+part.split("_")[1])
      label_annotation_LWK_copied = copy.deepcopy(label_annotation_LWK)

    threshold_for_choosing = random.random()

    if  threshold_for_choosing > 0.4 and part.split("_")[2] == 'Image':

      inner_threshold = random.random()

      if inner_threshold > 0.4 :
          to_replace_with,_ = random.choice(list(table_coordinates_LWK.items()))

          if to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1] != part.split("_")[0]+"_"+part.split("_")[1]:
            replacing_img_page = cv2.imread(image_path+"/"+to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1])
            #main_img_page = cv2.imread(image_path+"/"+part.split("_")[0])

            extract_roi(table_coordinates_LWK[to_replace_with],replacing_img_page,to_replace_with)
            extract_roi(image_coordinates_LWK[part],main_img_page,part)


            extracted_im_1 = cv2.imread(to_replace_with.split('/')[0]+"_"+to_replace_with.split('/')[1]+"_extracted.png")
            extracted_im_2 = cv2.imread(part.split('/')[0]+"_"+part.split('/')[1]+"_extracted.png") 
            white_img = np.zeros(extracted_im_2.shape,dtype=np.uint8)
            white_img.fill(255)
          
            if abs(extracted_im_1.shape[0]-extracted_im_2.shape[0])>45 or abs(extracted_im_1.shape[1]-extracted_im_2.shape[1])>45: # changed from 40 to 55
              # this will execute if to_replace_with table is not of the accepted shape
              main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)
              transformed_extracted_im_2 = linear_transformation(main_img_page,part,extracted_im_2)
              updated_main_img_page= replace(transformed_extracted_im_2,main_img_page,part,extracted_im_2)
              #cv2_imshow(updated_main_img_page)

              cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
              create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],part,part)
              continue

            main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)

            

            extracted_im_2 = linear_transformation(main_img_page,part,extracted_im_2)
            print("here in image to table")
            print(extracted_im_1.shape)
            print(extracted_im_2.shape)

            extracted_im_1,extracted_im_2 = change_size_white(extracted_im_1,extracted_im_2)

            print(extracted_im_1.shape)
            print(extracted_im_2.shape) 

            print("**********************")     

            # if aspect_ratio condition are acceptable
            #linear_transformation(main_img_page,part,extracted_im_1) # as ex_im_1 is now as same shape of ex_im_2
            updated_main_img_page= replace(extracted_im_1,main_img_page,part,extracted_im_2)
            #cv2_imshow(updated_main_img_page)

            cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
            create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],to_replace_with,part)

      else:
        to_replace_with,_ = random.choice(list(image_coordinates_LWK.items()))

        if to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1] != part.split("_")[0]+"_"+part.split("_")[1]:
          replacing_img_page = cv2.imread(image_path+"/"+to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1])
          #main_img_page = cv2.imread(image_path+"/"+part.split("_")[0])

          extract_roi(image_coordinates_LWK[to_replace_with],replacing_img_page,to_replace_with)
          extract_roi(image_coordinates_LWK[part],main_img_page,part)


          extracted_im_1 = cv2.imread(to_replace_with.split('/')[0]+"_"+to_replace_with.split('/')[1]+"_extracted.png")
          extracted_im_2 = cv2.imread(part.split('/')[0]+"_"+part.split('/')[1]+"_extracted.png") 
          white_img = np.zeros(extracted_im_2.shape,dtype=np.uint8)
          white_img.fill(255)

          if abs(extracted_im_1.shape[0]-extracted_im_2.shape[0])>45 or abs(extracted_im_1.shape[1]-extracted_im_2.shape[1])>45:
              # this will execute if to_replace_with image is not of the accepted shape
              main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)
              transformed_extracted_im_2 = linear_transformation(main_img_page,part,extracted_im_2)
              updated_main_img_page= replace(transformed_extracted_im_2,main_img_page,part,extracted_im_2)
              #cv2_imshow(updated_main_img_page)

              cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
              create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],part,part)
              continue

          main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)

          extracted_im_2 = linear_transformation(main_img_page,part,extracted_im_2)

          print("here in image to image")
          print(extracted_im_1.shape)
          print(extracted_im_2.shape)

          extracted_im_1,extracted_im_2 = change_size_white(extracted_im_1,extracted_im_2)

          print(extracted_im_1.shape)
          print(extracted_im_2.shape)      
          print("**********************")
          updated_main_img_page= replace(extracted_im_1,main_img_page,part,extracted_im_2)
          #cv2_imshow(updated_main_img_page)

          cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
          create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],to_replace_with,part)

    elif threshold_for_choosing > 0.35 and part.split("_")[2] == 'Table':

      to_replace_with,_ = random.choice(list(table_coordinates_LWK.items()))

      if to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1] != part.split("_")[0]+"_"+part.split("_")[1]:
        replacing_img_page = cv2.imread(image_path+"/"+to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1])
        #main_img_page = cv2.imread(image_path+"/"+part.split("_")[0])

        extract_roi(table_coordinates_LWK[to_replace_with],replacing_img_page,to_replace_with)
        extract_roi(table_coordinates_LWK[part],main_img_page,part)


        extracted_im_1 = cv2.imread(to_replace_with.split('/')[0]+"_"+to_replace_with.split('/')[1]+"_extracted.png")
        extracted_im_2 = cv2.imread(part.split('/')[0]+"_"+part.split('/')[1]+"_extracted.png") 
        white_img = np.zeros(extracted_im_2.shape,dtype=np.uint8)
        white_img.fill(255)

        if abs(extracted_im_1.shape[0]-extracted_im_2.shape[0])>40 or abs(extracted_im_1.shape[1]-extracted_im_2.shape[1])>40:
              # this will execute if to_replace_with table is not of the accepted shape
              main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)
              transformed_extracted_im_2 = linear_transformation(main_img_page,part,extracted_im_2)
              updated_main_img_page= replace(transformed_extracted_im_2,main_img_page,part,extracted_im_2)
              #cv2_imshow(updated_main_img_page)

              cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
              create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],part,part)
              continue

        main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)

        extracted_im_2 = linear_transformation(main_img_page,part,extracted_im_2)

        print("here in table to table")
        print(extracted_im_1.shape)
        print(extracted_im_2.shape)

        extracted_im_1,extracted_im_2 = change_size_white(extracted_im_1,extracted_im_2)

        print(extracted_im_1.shape)
        print(extracted_im_2.shape)      
        print("**********************")
        updated_main_img_page= replace(extracted_im_1,main_img_page,part,extracted_im_2)
        #cv2_imshow(updated_main_img_page)

        cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
        create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],to_replace_with,part)

    elif threshold_for_choosing > 0.5 and part.split("_")[2] == 'Math':

      to_replace_with,_ = random.choice(list(math_coordinates_LWK.items()))

      if to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1] != part.split("_")[0]+"_"+part.split("_")[1]:
        replacing_img_page = cv2.imread(image_path+"/"+to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1])
        #main_img_page = cv2.imread(image_path+"/"+part.split("_")[0])

        extract_roi(math_coordinates_LWK[to_replace_with],replacing_img_page,to_replace_with)
        extract_roi(math_coordinates_LWK[part],main_img_page,part)


        extracted_im_1 = cv2.imread(to_replace_with.split('/')[0]+"_"+to_replace_with.split('/')[1]+"_extracted.png")
        extracted_im_2 = cv2.imread(part.split('/')[0]+"_"+part.split('/')[1]+"_extracted.png") 
        white_img = np.zeros(extracted_im_2.shape,dtype=np.uint8)
        white_img.fill(255)

        if abs(extracted_im_1.shape[0]-extracted_im_2.shape[0])>40 or abs(extracted_im_1.shape[1]-extracted_im_2.shape[1])>40:
          continue

        main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)

        print("here in math to math")
        print(extracted_im_1.shape)
        print(extracted_im_2.shape)

        extracted_im_1,extracted_im_2 = change_size_white(extracted_im_1,extracted_im_2)

        print(extracted_im_1.shape)
        print(extracted_im_2.shape)      
        print("**********************")
        updated_main_img_page= replace(extracted_im_1,main_img_page,part,extracted_im_2)
        #cv2_imshow(updated_main_img_page)

        cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
        create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],to_replace_with,part)
  
    if  threshold_for_choosing > 0.4 and part.split("_")[2] == 'Text':

      inner_threshold = random.random()

      if inner_threshold > 0.4 :
          to_replace_with,_ = random.choice(list(text_coordinates_LWK.items()))

          if to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1] != part.split("_")[0]+"_"+part.split("_")[1]:
            replacing_img_page = cv2.imread(image_path+"/"+to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1])
            #main_img_page = cv2.imread(image_path+"/"+part.split("_")[0])

            extract_roi(text_coordinates_LWK[to_replace_with],replacing_img_page,to_replace_with)
            extract_roi(text_coordinates_LWK[part],main_img_page,part)


            extracted_im_1 = cv2.imread(to_replace_with.split('/')[0]+"_"+to_replace_with.split('/')[1]+"_extracted.png")
            extracted_im_2 = cv2.imread(part.split('/')[0]+"_"+part.split('/')[1]+"_extracted.png") 
            white_img = np.zeros(extracted_im_2.shape,dtype=np.uint8)
            white_img.fill(255)
          
            if abs(extracted_im_1.shape[0]-extracted_im_2.shape[0])>30 or abs(extracted_im_1.shape[1]-extracted_im_2.shape[1])>30: # changed from 40 to 55
                continue

            main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)

            #extracted_im_2 = linear_transformation(main_img_page,part,extracted_im_2)
            print("here in text to text")
            print(extracted_im_1.shape)
            print(extracted_im_2.shape)

            extracted_im_1,extracted_im_2 = change_size_white(extracted_im_1,extracted_im_2)

            print(extracted_im_1.shape)
            print(extracted_im_2.shape) 

            print("**********************")     

            # if aspect_ratio condition are acceptable
            #linear_transformation(main_img_page,part,extracted_im_1) # as ex_im_1 is now as same shape of ex_im_2
            updated_main_img_page= replace(extracted_im_1,main_img_page,part,extracted_im_2)
            #cv2_imshow(updated_main_img_page)

            cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
            create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],to_replace_with,part)

      else:
        to_replace_with,_ = random.choice(list(math_coordinates_LWK.items()))

        if to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1] != part.split("_")[0]+"_"+part.split("_")[1]:
          replacing_img_page = cv2.imread(image_path+"/"+to_replace_with.split("_")[0]+"_"+to_replace_with.split("_")[1])
          #main_img_page = cv2.imread(image_path+"/"+part.split("_")[0])

          extract_roi(math_coordinates_LWK[to_replace_with],replacing_img_page,to_replace_with)
          extract_roi(text_coordinates_LWK[part],main_img_page,part)


          extracted_im_1 = cv2.imread(to_replace_with.split('/')[0]+"_"+to_replace_with.split('/')[1]+"_extracted.png")
          extracted_im_2 = cv2.imread(part.split('/')[0]+"_"+part.split('/')[1]+"_extracted.png") 
          white_img = np.zeros(extracted_im_2.shape,dtype=np.uint8)
          white_img.fill(255)

          if abs(extracted_im_1.shape[0]-extracted_im_2.shape[0])>30 or abs(extracted_im_1.shape[1]-extracted_im_2.shape[1])>30:
             continue

          main_img_page = put_white_img(white_img,main_img_page,part,extracted_im_2)

          #extracted_im_2 = linear_transformation(main_img_page,part,extracted_im_2)

          print("here in text to math")
          print(extracted_im_1.shape)
          print(extracted_im_2.shape)

          extracted_im_1,extracted_im_2 = change_size_white(extracted_im_1,extracted_im_2)

          print(extracted_im_1.shape)
          print(extracted_im_2.shape)      
          print("**********************")
          updated_main_img_page= replace(extracted_im_1,main_img_page,part,extracted_im_2)
          #cv2_imshow(updated_main_img_page)

          cv2.imwrite(os.path.join(new_dir ,page_name.split(".")[0]+"_"+str(number)+".jpeg"), updated_main_img_page)
          create_bbx(page_name.split(".")[0]+"_"+str(number)+".jpeg",updated_main_img_page,label_annotation_LWK_copied[part],to_replace_with,part)

    count = count+1
  #break

In [27]:
image_path = dataset_path+"/train"

In [ ]:
#process_image('GK2_page-0182.jpg',number)

In [30]:
for i in range(10):
  page_name = random.choice(image_list)
  from_random_page = page_details_LWK[page_name][0]
  text_count = 0
  #image_count = 0
  #table_count = 0
  #math_count = 0
  total_count = len(label_annotation_LWK[from_random_page])
  for i in range(total_count):
    j=label_annotation_LWK[from_random_page][i]['label']
    if j=='Text':
      text_count+=1
  if (text_count/total_count)*100 >= 70:
    continue 
  process_image(page_name,number)
  number+=1

GON2/GK2_page-0109.jpg_Text_0&0
GON2_GK2_page-0249.jpg_Text_9
292 585 552 628
GON2_GK2_page-0109.jpg_Text_0
325 77 405 104
GON2/GK2_page-0109.jpg_Text_1&0
GON2_GK2_page-0033.jpg_Text_4
121 314 633 346
GON2_GK2_page-0109.jpg_Text_1
262 126 467 159
GON2/GK2_page-0109.jpg_Image_2&0
GON2/GK2_page-0109.jpg_Text_3&1
GON2/GK2_page-0109.jpg_Text_4&2
GON2_GK2_page-0007.jpg_Text_16
169 604 562 633
GON2_GK2_page-0109.jpg_Text_4
76 455 250 483
GON2/GK2_page-0109.jpg_Text_5&2
GON2_GK2_page-0125.jpg_Text_22_3
503 1097 622 1128
GON2_GK2_page-0109.jpg_Text_5
116 493 475 524
GON2/GK2_page-0109.jpg_Text_6&2
GON2_GK2_page-0063.jpg_Text_19
209 1012 592 1056
GON2_GK2_page-0109.jpg_Text_6
77 557 531 583
GON2/GK2_page-0109.jpg_Text_7&2
GON2_GK2_page-0313.jpg_Text_15
228 847 488 885
GON2_GK2_page-0109.jpg_Text_7
77 611 658 646
GON2/GK2_page-0109.jpg_Text_9&2
GON2_GK2_page-0234.jpg_Text_20
122 1137 581 1171
GON2_GK2_page-0109.jpg_Text_9
77 689 660 724
GON2/GK2_page-0109.jpg_Text_10&2
GON2_GK2_page-0198.jpg_Mat

In [31]:
print(number)

3


In [32]:
temp_dir = "checking_layout"
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

In [33]:
# The purpose of this function is to check the images generated 
def create_for_all():

  image_path = "generated_layout/dataset/train"

  for i in bbx_dict['images']:
    page_image = cv2.imread(image_path+"/"+i)
    dh, dw, _ = page_image.shape
    for j in range(len(bbx_dict['images'][i]['annotations'])):
      #print(bbx_dict['images'][i]['annotations'][j]['bbox']," ","label = ",bbx_dict['images'][i]['annotations'][j]['label'])  
      x1 = int( bbx_dict['images'][i]['annotations'][j]['bbox']['xmin']*dw)
      y1 = int( bbx_dict['images'][i]['annotations'][j]['bbox']['ymin']*dh)
      x2 = int( bbx_dict['images'][i]['annotations'][j]['bbox']['xmax']*dw)#end_cell[0]+end_cell[3]
      y2 = int( bbx_dict['images'][i]['annotations'][j]['bbox']['ymax']*dh)#end_cell[1]+end_cell[2]
      cv2.rectangle(page_image, (x1, y1), (x2, y2), (0,0,255), 2)
      center = ((x1+x2)//2,(y1+y2)//2)
      cv2.putText(page_image,bbx_dict['images'][i]['annotations'][j]['label'], center, cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 2)
      cv2.imwrite("checking_layout/"+i, page_image)
    #cv2_imshow(page_image)
    #breakkkkkkkk
  # print(x," : ",l)
  #print(x1,y1,x2,y2)jfjd
  #bb_coordinates[img_name] = [x1,y1,x2,y2]
  #print(end_y)
  #table_image = imutils.resize(table_image,width = 2964) #previously it was 2564

  #ROI = page_image[y1:y2, x1:x2]    
  # draw the bounding box on the image
  #cv2.rectangle(page_image, (x1, y1), (x2, y2), (0,0,255), 2)
  #cv2.imwrite(img_name_modified+'_extracted.png', ROI)

In [34]:
create_for_all()

In [ ]:
#!zip -r /content/CHECKING_LAYOUT_FINAL.zip /content/checking_layout

In [ ]:
len(bbx_dict['images'])

4045

In [ ]:
#for i in range(len(bbx_dict['images']['GK2_page-0123_801.jpeg']['annotations'])):
#  print(bbx_dict['images']['GK2_page-0123_801.jpeg']['annotations'][i]['label'])

In [ ]:
#!zip -r /content/GEN_LAY_final.zip /content/generated_layout

In [ ]:
with open('GENERATED_GEN_LAY_FINAL_corrected_fully.bbx', 'w') as file:
     file.write(json.dumps(bbx_dict))
  
#os.path.join('generated_layout/dataset','generated_LWK.bbx' )